In [1]:
! nvidia-smi

Sun Mar  5 06:20:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  Off |
|  0%   32C    P8    16W / 480W |    791MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import re
import gc
import pdb
import sys
import json
import math
import time
import wandb
import pickle
import shutil
import joblib
import random
import pathlib
import requests
import warnings
from glob import glob
from typing import List
from pathlib import Path
from tqdm.auto import tqdm
from pandarallel import pandarallel

import scipy
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import (
    StratifiedKFold,
    KFold,
    GroupKFold,
    StratifiedGroupKFold
)
from sklearn.metrics import mean_squared_error, f1_score, fbeta_score, recall_score, precision_score
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.decomposition import TruncatedSVD

import xgboost as xgb
import lightgbm as lgb

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.checkpoint import checkpoint
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F

from pytorch_metric_learning import losses

import tokenizers
import sentencepiece
import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer
from transformers import AdamW
from transformers import logging
from transformers import DataCollatorWithPadding

import cupy as cp
from cuml import NearestNeighbors
from kaggle.api.kaggle_api_extended import KaggleApi

sys.path.append("/home/working/")
from kagglib.utils.utils import  Timer, reduce_mem_usage, get_logger, decorate, setup, dataset_create_new
from kagglib.utils.exp_manage import set_wandb
from kagglib.tabular.blocks import AbstractBaseBlock, IdentityBlock, LabelEncodingBlock, SVDBlock, run_blocks
from kagglib.tabular.model_selection import train_cv, predict_cv
from kagglib.tabular.model import XGBoost
from kagglib.tabular.model import get_model, save_model, load_model
from kagglib.nlp.preprocessing import resolve_encodings_and_normalize
from kagglib.nlp.model import (
    AttentionPooling,
    MeanPooling,
    WeightedLayerPooling,
    freeze,
    replace_mixout,
    reinit_bert,
)
from kagglib.nlp.activation import softmax, sigmoid
from kagglib.nlp.optimizer import (
    get_scheduler,
    get_optimizer_grouped_parameters,
)

%load_ext autoreload
%autoreload 2
%env TOKENIZERS_PARALLELISM=true

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 300)
pandarallel.initialize(progress_bar=True)
plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('ggplot')

logging.set_verbosity_error()
logging.set_verbosity_warning()

env: TOKENIZERS_PARALLELISM=true
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Setup & data load

In [3]:
def fbeta_wrapper(y_true, y_pred):
    beta = 2
    return fbeta_score(y_true, y_pred, beta)

class Config:
    AUTHOR = "shu421"

    EXP = "exp025"
    COMPETITION = "learning-equality-curriculum-recommendations"
    DATASET_PATH = []
    BASE_PATH = "/home/working/"
    api_path = "/root/.kaggle/kaggle.json"

    # Language Model Config
    MODEL_PATH = "sentence-transformers/all-MiniLM-L6-v2"
    # MODEL_PATH = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

    # train
    apex=True
    seed = 42
    num_fold = 5
    train_fold = [0]
    batch_size = 768
    n_epoch = 10
    max_len = 256
    num_classes = 1


    # optimizer
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    weight_decay = 0.01
    betas = (0.9, 0.999)
    lr_weight_decay = 0.95

    # scheduler
    scheduler = "cosine"
    min_lr = 1e-6
    eps = 1e-6
    eval_step = 4000
    num_cycles = 0.5
    num_warmup_steps_rate = 0.1
    clip_grad_norm = 1000

    # gradient accumulation
    gradient_accumulation_steps = 1

    es_patience = 3


    # weight and bias
    wandb = False

    # GPU Optimize Settings
    gpu_optimize_config= {
        "freezing": False,
        "gradient_checkpoint": True
    }

    # GBDT
    gbdt_model = "XGBoost"
    stopping_rounds = 50
    log_evaluation = 500
    model_params = {
        "objective": "binary:logistic",
        # "eval_metric": fbeta_wrapper,
        "eval_metric": "logloss",
        "learning_rate": 0.3,
        "tree_method": "gpu_hist",
        "random_state": seed,
        "n_estimators": 99999,
    }
    train_params = {
        "verbose": log_evaluation,
    }

    upload_from_colab = True

# setup
cfg = setup(Config)

In [4]:
class Metrics_Config(Config):
    AUTHOR = "shu421"

    MODEL_PATH = "sentence-transformers/all-MiniLM-L6-v2"
    # MODEL_PATH = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    MODEL_CONFIG_PATH = MODEL_PATH

    # train
    apex=True
    seed = 42
    num_fold = 5
    train_fold = [0]
    batch_size = 1024
    n_epoch = 30
    max_len = 128
    num_classes = 1


    # optimizer
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    weight_decay = 0.01
    betas = (0.9, 0.999)
    lr_weight_decay = 0.95

    # scheduler
    scheduler="cosine"
    min_lr = 1e-6
    eps = 1e-6
    eval_step = 400
    num_cycles=0.5
    num_warmup_steps_rate=0.1
    clip_grad_norm = 1000

    # gradient accumulation
    gradient_accumulation_steps = 1

    es_patience = 3

    # weight and bias
    wandb = False

    # GPU Optimize Settings
    gpu_optimize_config= {
        "freezing": False,
        "gradient_checkpoint": True
    }

metric_cfg = setup(Metrics_Config)

In [5]:
# set log functions
LOGGER = get_logger(cfg.OUTPUT_EXP)
log_filepath = os.path.join(cfg.OUTPUT, f"{cfg.EXP}.log")
# if os.path.isfile(log_filepath):
#     with open(log_filepath, "w") as f:
#         pass
#     f.close()
if cfg.wandb:
    run = set_wandb(cfg, name=cfg.EXP, group=cfg.MODEL_PATH, config_path="/root/.kaggle/wandb.json")
if metric_cfg.wandb:
    run = set_wandb(cfg, name=cfg.EXP + "_metric", group=metric_cfg.MODEL_PATH, config_path="/root/.kaggle/wandb.json")

# Metrics

In [6]:
def comp_fbeta_score(y_true_ids: pd.Series, y_pred_ids: pd.Series, beta=2, eps=1e-15):
    true_ids = y_true_ids.str.split()
    pred_ids = y_pred_ids.str.split()
    score_list = []
    for true, pred in zip(true_ids.tolist(), pred_ids.tolist()):
        TP = (set(true) & set(pred))
        precision = len(TP) / (len(pred))
        recall = len(TP) / len(true)
        f2 = (1+beta**2) * (precision*recall) / ((beta**2)*precision+recall+eps)
        score_list.append(f2)
    score = sum(score_list) / len(score_list)
    return score

def comp_recall_score(y_true_ids: pd.Series, y_pred_ids: pd.Series, beta=2, eps=1e-15):
    true_ids = y_true_ids.str.split()
    pred_ids = y_pred_ids.str.split()
    score_list = []
    for true, pred in zip(true_ids.tolist(), pred_ids.tolist()):
        TP = (set(true) & set(pred))
        recall = len(TP) / len(true)

        score_list.append(recall)
    score = sum(score_list) / len(score_list)
    return score

def calc_comp_score(train_df, oof, correlation_df, thr=0.1):
    """2値分類の予測からcvを計算する"""
    oof_preds = np.where(oof>=thr, 1, 0)
    pred_df = train_df[oof_preds==1]
    pred_df = pred_df.groupby("topic_id")["content_id"].apply(list).apply(" ".join)
    pred_df = pd.merge(correlation_df[["topic_id"]], pred_df, on="topic_id", how="left")
    pred_df = pred_df.fillna("")

    cv_score = comp_fbeta_score(correlation_df["content_id"], pred_df["content_id"])
    return cv_score


import heapq
from dataclasses import dataclass

import torch
from typing import Callable
import numpy as np

def cos_sim(a, b):
    # From https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/util.py#L31
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))


# From: https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/util.py#L204
def semantic_search(
    query_embeddings: torch.Tensor,
    corpus_embeddings: torch.Tensor,
    query_chunk_size: int = 100,
    corpus_chunk_size: int = 500000,
    top_k: int = 10,
    score_function: Callable[[torch.Tensor, torch.Tensor], torch.Tensor] = cos_sim,
):
    """
    This function performs a cosine similarity search between a list of query embeddings  and a list of corpus embeddings.
    It can be used for Information Retrieval / Semantic Search for corpora up to about 1 Million entries.
    :param query_embeddings: A 2 dimensional tensor with the query embeddings.
    :param corpus_embeddings: A 2 dimensional tensor with the corpus embeddings.
    :param query_chunk_size: Process 100 queries simultaneously. Increasing that value increases the speed, but requires more memory.
    :param corpus_chunk_size: Scans the corpus 100k entries at a time. Increasing that value increases the speed, but requires more memory.
    :param top_k: Retrieve top k matching entries.
    :param score_function: Function for computing scores. By default, cosine similarity.
    :return: Returns a list with one entry for each query. Each entry is a list of dictionaries with the keys 'corpus_id' and 'score', sorted by decreasing cosine similarity scores.
    """

    if isinstance(query_embeddings, (np.ndarray, np.generic)):
        query_embeddings = torch.from_numpy(query_embeddings)
    elif isinstance(query_embeddings, list):
        query_embeddings = torch.stack(query_embeddings)

    if len(query_embeddings.shape) == 1:
        query_embeddings = query_embeddings.unsqueeze(0)

    if isinstance(corpus_embeddings, (np.ndarray, np.generic)):
        corpus_embeddings = torch.from_numpy(corpus_embeddings)
    elif isinstance(corpus_embeddings, list):
        corpus_embeddings = torch.stack(corpus_embeddings)

    # Check that corpus and queries are on the same device
    if corpus_embeddings.device != query_embeddings.device:
        query_embeddings = query_embeddings.to(corpus_embeddings.device)

    queries_result_list = [[] for _ in range(len(query_embeddings))]

    for query_start_idx in range(0, len(query_embeddings), query_chunk_size):
        # Iterate over chunks of the corpus
        for corpus_start_idx in range(0, len(corpus_embeddings), corpus_chunk_size):
            # Compute cosine similarities
            cos_scores = score_function(
                query_embeddings[query_start_idx : query_start_idx + query_chunk_size],
                corpus_embeddings[
                    corpus_start_idx : corpus_start_idx + corpus_chunk_size
                ],
            )

            # Get top-k scores
            cos_scores_top_k_values, cos_scores_top_k_idx = torch.topk(
                cos_scores,
                min(top_k, len(cos_scores[0])),
                dim=1,
                largest=True,
                sorted=False,
            )
            cos_scores_top_k_values = cos_scores_top_k_values.cpu().tolist()
            cos_scores_top_k_idx = cos_scores_top_k_idx.cpu().tolist()

            for query_itr in range(len(cos_scores)):
                for sub_corpus_id, score in zip(
                    cos_scores_top_k_idx[query_itr], cos_scores_top_k_values[query_itr]
                ):
                    corpus_id = corpus_start_idx + sub_corpus_id
                    query_id = query_start_idx + query_itr
                    if len(queries_result_list[query_id]) < top_k:
                        heapq.heappush(
                            queries_result_list[query_id], (score, corpus_id)
                        )  # heaqp tracks the quantity of the first element in the tuple
                    else:
                        heapq.heappushpop(
                            queries_result_list[query_id], (score, corpus_id)
                        )

    # change the data format and sort
    for query_id in range(len(queries_result_list)):
        for doc_itr in range(len(queries_result_list[query_id])):
            score, corpus_id = queries_result_list[query_id][doc_itr]
            queries_result_list[query_id][doc_itr] = {
                "corpus_id": corpus_id,
                "score": score,
            }
        queries_result_list[query_id] = sorted(
            queries_result_list[query_id], key=lambda x: x["score"], reverse=True
        )

    return queries_result_list


def get_topk_preds(topic_embeddings, content_embeddings, df, k=100, return_idxs=True):
    """
    df has the same number of rows as topic_embeddings and content_embeddings.
    A bunch of the topic embeddings are duplicates, so we'll dedupe before finding 
    nearest neighbors.

    Returns tuple (prediction content ids, topic ids)

        prediction content ids is a list of lists. The outer list has the same number of elements as unique topic
        ids. The inner list contains content ids and the length is equal to k (num nearest neighbors).
    """

    content_ids = df.content_id.values

    # These idx values will be used to compare with the idx
    # values returned by `semantic_search` to calculate recall.
    df["idx"] = list(range(len(content_embeddings)))

    deduped = df[["idx", "topic_id"]].drop_duplicates("topic_id")
    topic_embeddings = topic_embeddings[deduped["idx"]]
    device = torch.device("cuda:0")

    # Compare each of the topic embeddings to each of the
    # content embeddings and return a ranking for each one.
    # Works much, much faster on GPU.
    search_results = semantic_search(
        torch.tensor(topic_embeddings, device=device),
        torch.tensor(content_embeddings, device=device),
        top_k=k,
    )

    # `search_results` is a list of lists. The inner list
    # has a `dict` at each element.
    # The dict has two keys: `corpus_id` and `score`.
    all_pred_c_ids = [[content_ids[x["corpus_id"]] for x in row] for row in search_results]

    return all_pred_c_ids, deduped["topic_id"].tolist()

def Precision_score(pred_content, gt_content):
    """
    Arguments can be int (idx) or string values.
    """
    def precision(pred, gt):
        tp = len(set(pred)&set(gt))
        fp = len(set(pred)-set(gt))
        return tp/(tp+fp)

    # Get a recall score for each row of the dataset
    return [
        precision(pred, gt)
        for pred, gt in zip(pred_content, gt_content)
    ]

def Recall_score(pred_content, gt_content):
    """
    Arguments can be int (idx) or string values.
    """
    def recall(pred, gt):
        tp = len(set(pred)&set(gt))
        return tp/len(set(gt))

    # Get a recall score for each row of the dataset
    return [
        recall(pred, gt)
        for pred, gt in zip(pred_content, gt_content)
    ]

def mean_f2_score(Precision_scores, Recall_scores):
    """
    Inputs should be outputs of the `Precision_score` and 
    `Recall_score` functions.
    """
    beta = 2

    def f2_score(precision, recall):
        return (1+beta**2)*(precision*recall)/(beta**2*precision+recall+1e-7)

    return round(np.mean([f2_score(p, r) for p, r in zip(Precision_scores, Recall_scores)]), 5)



def compute_metrics(topic_embeddings, content_embeddings, val_df, k=100):
    """
    After creating embeddings for all of the topic and content texts,
    perform a semantic search and measure the recall@100. The model
    has not seen any of these examples before, so it should be a
    good measure of how well the model can generalize.

    Since the dataset uses the exploded view of the correlations
    (one topic with 5 contents is 5 rows), I need to deduplicate
    the topic embeddings. Then I can use the `semantic_search`
    function taken from the sentence-transformers util to
    do a cosine similarity search of the topic embeddings with all
    content embeddings. This function conveniently returns the top
    `k` indexes, which makes it easy to compare with the true indexes.
    """

    if isinstance(k, int):
        k = [k]

    # eval_predictions is a tuple of (model_output, labels)
    # The model_output is whatever is returned by `compute_loss`

    pred_content_ids, topic_ids = get_topk_preds(topic_embeddings, content_embeddings, val_df, k=max(k))

    # Make sure true content ids are in same order as predictions
    grouped = val_df[["topic_id", "content_id"]].groupby("topic_id").agg(list)
    true_content_ids = grouped.loc[topic_ids].reset_index()["content_id"]

    metrics = {}
    for kk in k:
        top_preds = [row[:kk] for row in pred_content_ids]
        precisions = Precision_score(top_preds, true_content_ids)
        recalls = Recall_score(top_preds, true_content_ids)
        f2 = mean_f2_score(precisions, recalls)

        metrics[f"recall@{kk}"] = np.round(np.mean(recalls), 5)
        metrics[f"f2@{kk}"] = np.round(f2, 5)

    return metrics

def get_StratifiedGroupKFold_list(X, y=None, groups=None, n_splits=5, seed=42):
    """cv_listを取得"""
    cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    cv_list = list(cv.split(X, y, groups))
    return cv_list

def get_StratifiedKFold_list(X, y=None, groups=None, n_splits=5, seed=42):
    """cv_listを取得"""
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    cv_list = list(cv.split(X, y))
    return cv_list

def get_GroupKFold_list(X, y=None, groups=None, n_splits=5, seed=42):
    cv = GroupKFold(n_splits=n_splits)
    cv_list = list(cv.split(X, y, groups=groups))
    return cv_list


In [7]:
def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

In [8]:
def del_newline(text):
    """改行を削除"""
    text = text.replace("\n\n", " ").replace("\n", " ").replace("\r", " ").replace("\f", " ").replace("\t", " ").replace("\v", " ")
    return text

def del_url_str(text):
    """URLに含まれる必要のない文字列を削除"""
    text = text.replace("https", "").replace("http", "").replace("www", "")
    return text

def del_symbol(text):
    """記号を削除"""
    # text = re.sub('[\[\]\!\"\#\$\%\&\'\\\\\(\)\*\+\-\/\:\;\<\=\>\@\^\_\`\{\|\}\~\「\」\〔\〕\“\”\〈\〉\『\』\【\】\＆\＊\・\（\）\＄\＃\＠\。\、\！\｀\＋\￥\％]', " ", text)
    # 複数回繰り返す文字列を削除
    text = re.sub("\.{2,}", " ", text)
    text = re.sub("(\. ){2,}", " ", text)
    text = re.sub("(\ .){2,}", " ", text)
    text = re.sub("\-{2,}", " ", text)
    text = re.sub("\|{2,}", " ", text)
    text = re.sub("\!{2,}", " ", text)
    text = re.sub("\?{2,}", " ", text)
    return text

def lower_text(text):
    """小文字化"""
    return text.lower()

def normalize_unicode(text, form='NFKC'):
    """正規化"""
    import unicodedata
    normalized_text = unicodedata.normalize(form, text)
    return normalized_text

def clean_text(text):
    # text = lower_text(text)
    # text = del_newline(text)
    # text = del_url_str(text)
    # text = del_symbol(text)
    # text = normalize_unicode(text)
    return text

In [9]:
def get_whole_df():
    content_df = pd.read_csv(os.path.join(cfg.INPUT, 'content.csv'), usecols=["id", "title", "description", "kind", "text", "language"])
    topic_df = pd.read_csv(os.path.join(cfg.INPUT, 'topics.csv'))
    correlation_df = pd.read_csv(os.path.join(cfg.INPUT, 'correlations.csv'))
    sub_df = pd.read_csv(os.path.join(cfg.INPUT, 'sample_submission.csv'))
    return content_df, topic_df, correlation_df, sub_df

def preprocess_df(content_df, topic_df, correlation_df):
    content_df = content_df.add_prefix("content_")
    topic_df = topic_df.add_prefix("topic_")
    correlation_df = correlation_df.rename(columns={"content_ids":"content_id"})
    content_df = content_df.fillna("")
    topic_df = topic_df.fillna("")
    # cleaning
    for col in ["topic_title", "topic_description"]:
        topic_df[col] = topic_df[col].apply(clean_text)
    for col in ["content_title", "content_description", "content_kind", "content_text"]:
        content_df[col] = content_df[col].apply(clean_text)
    return content_df, topic_df, correlation_df

def get_processed_df():
    content_df, topic_df, correlation_df, sub_df = get_whole_df()
    content_df, topic_df, correlation_df = preprocess_df(content_df, topic_df, correlation_df)
    return content_df, topic_df, correlation_df, sub_df

In [10]:
# get context
sys.path.append("../input/")
from helper_funcs import Topic, ContentItem


def get_topic_context(x):
    topic = Topic(content_df, topic_df, correlation_df, [x])
    topic_context = topic.get_breadcrumbs(separator=" ")
    return topic_context

# 1st Stage Functions

In [11]:
def create_topic_emb_feature_df(topic_df):
    output_df = topic_df[["topic_id", "topic_channel", "topic_category"]].copy()
    output_df["topic_text"] = topic_df["topic_title"] + " [SEP] " + topic_df["topic_description"] + " [SEP] " + topic_df["topic_context"]
    return output_df

def create_content_emb_feature_df(content_df):
    output_df = content_df[["content_id"]].copy()
    output_df["content_text"] = content_df["content_title"] + " [SEP] " + content_df["content_description"] + " [SEP] " + content_df["content_kind"] + " [SEP] " + content_df["content_text"]
    return output_df

In [12]:
class MetricDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.topic_text = df["topic_text"].to_numpy()
        self.content_text = df["content_text"].to_numpy()
        self.label = df["target"].to_numpy()

    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):
        topic_text = self.prepare_input(self.cfg, self.topic_text[index])
        content_text = self.prepare_input(self.cfg, self.content_text[index])
        label = torch.tensor(self.label[index], dtype=torch.float)
        return topic_text, content_text, label

    @staticmethod
    def prepare_input(cfg, text):
        inputs = cfg.tokenizer(
            text,
            add_special_tokens=True,
            max_length=cfg.max_len,
            padding="max_length",
            truncation=True,
            return_offsets_mapping=False
            )
        inputs['input_ids'] = torch.tensor(
            inputs['input_ids'],
            dtype=torch.long
        )
        inputs['attention_mask'] = torch.tensor(
            inputs['attention_mask'],
            dtype=torch.long
        )
        inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
        }
        return inputs

class MetricModel(nn.Module):
    def __init__(self, cfg): 
        super().__init__()
        self.cfg = cfg
        self.gpu_optimize_config = cfg.gpu_optimize_config
        self.config = AutoConfig.from_pretrained(
            cfg.MODEL_PATH,
            output_hidden_states=True
        )
        cfg.hidden_size = self.config.hidden_size
        self.config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout": 0.,
                "hidden_dropout_prob": 0.,
                "attention_dropout": 0.,
                "attention_probs_dropout_prob": 0,
            }
        )
        self.model = AutoModel.from_pretrained(
            cfg.MODEL_PATH,
            config=self.config
        )
        self.model.resize_token_embeddings(len(self.cfg.tokenizer))

        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, cfg.num_classes)
        self._init_weights(self.fc)
        self.ln = nn.LayerNorm(self.config.hidden_size)
        self._init_weights(self.ln)

        self.drop1 = nn.Dropout(0.1)
        self.drop2 = nn.Dropout(0.2)
        self.drop3 = nn.Dropout(0.3)
        self.drop4 = nn.Dropout(0.4)
        self.drop5 = nn.Dropout(0.5)

        # Freeze
        if self.gpu_optimize_config['freezing']:
            freeze(self.model.encoder.layer[:4])

        # Gradient Checkpointing
        if self.gpu_optimize_config['gradient_checkpoint']:
            self.model.gradient_checkpointing_enable()

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_state = outputs[0]
        feature = self.pool(last_hidden_state, inputs['attention_mask'])
        return feature
    
    def forward_classification(self, inputs):
        feature = self.forward_once(inputs)
        logits = self.fc(feature)
        return logits

    def forward_once(self, inputs):
        feature = self.feature(inputs)
        # feature = self.ln(feature)
        return feature

    def forward(self, inputs1, inputs2):
        # batch, hidden_size
        feature1 = self.forward_once(inputs1)
        feature2 = self.forward_once(inputs2)
        return feature1, feature2

In [13]:
class MultipleNegativesRankingLoss(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.loss_function = torch.nn.CrossEntropyLoss()

    def forward(self, embeddings_a, embeddings_b, labels=None):
        """
        Compute similarity between `a` and `b`.
        Labels have the index of the row number at each row. 
        This indicates that `a_i` and `b_j` have high similarity 
        when `i==j` and low similarity when `i!=j`.
        """

        similarity_scores = (
            cos_sim(embeddings_a, embeddings_b) * 20.0
        )  # Not too sure why to scale it by 20: https://github.com/UKPLab/sentence-transformers/blob/b86eec31cf0a102ad786ba1ff31bfeb4998d3ca5/sentence_transformers/losses/MultipleNegativesRankingLoss.py#L57

        labels = torch.tensor(
            range(len(similarity_scores)),
            dtype=torch.long,
            device=similarity_scores.device,
        )  # Example a[i] should match with b[i]

        return self.loss_function(similarity_scores, labels)


In [14]:
def get_cand_train_fn(_cfg, train_loader, valid_loader, train_df, valid_df, criterion, optimizer, scheduler, model, fold, epoch, best_val_score, es_count):
    LOGGER.info(f'{"="*20} epoch{epoch} {"="*20}')
    train_losses = []
    train_nums = []
    model.train()
    scaler = GradScaler(enabled=_cfg.apex)
    with tqdm(train_loader, total=len(train_loader)) as pbar:
        for step, (topic_inputs, content_inputs, labels) in enumerate(pbar):
            topic_inputs = collate(topic_inputs)
            content_inputs = collate(content_inputs)
            for k, v in topic_inputs.items():
                topic_inputs[k] = v.to(_cfg.device)
            for k, v in content_inputs.items():
                content_inputs[k] = v.to(_cfg.device)
            labels = labels.to(_cfg.device)
            with autocast(enabled=_cfg.apex):
                topic_emb, content_emb = model(topic_inputs, content_inputs)
            loss = criterion(topic_emb, content_emb)

            pbar.set_postfix({
                'loss': loss.item(),
                'lr': scheduler.get_lr()[0]
            })
            train_losses.append(loss.item() * len(labels))
            train_nums.append(len(labels))

            if _cfg.gradient_accumulation_steps > 1:
                loss = loss / _cfg.gradient_accumulation_steps

            scaler.scale(loss).backward()

            if _cfg.clip_grad_norm is not None:
                # scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(
                    model.parameters(),
                    _cfg.clip_grad_norm
                )

            if (step+1) % _cfg.gradient_accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            if step % _cfg.eval_step == 0 and step != 0:
                best_val_score, val_score, es_count = get_cand_valid_fn(
                    _cfg,
                    valid_loader,
                    valid_df,
                    criterion,
                    model,
                    fold,
                    epoch,
                    step,
                    best_val_score,
                    es_count
                    )
                model.train()

            if _cfg.wandb:
                wandb.log({f"[fold{fold}] train_loss": loss.item(),
                        f"[fold{fold}] lr": scheduler.get_lr()[0]})
    train_loss = sum(train_losses)/sum(train_nums)
    return train_loss, es_count



def get_cand_valid_fn(_cfg, valid_loader, valid_df, criterion, model, fold, epoch, step, best_val_score, es_count):
    val_losses = []
    topic_embs = []
    content_embs = []
    val_nums = []
    model.eval()
    with torch.no_grad():
        with tqdm(valid_loader, total=len(valid_loader)) as pbar:
            for (topic_inputs, content_inputs, labels) in pbar:
                topic_inputs = collate(topic_inputs)
                content_inputs = collate(content_inputs)
                for k, v in topic_inputs.items():
                    topic_inputs[k] = v.to(_cfg.device)
                for k, v in content_inputs.items():
                    content_inputs[k] = v.to(_cfg.device)
                labels = labels.to(_cfg.device)
                with autocast(enabled=_cfg.apex):
                    topic_emb, content_emb = model(topic_inputs, content_inputs)
                loss = criterion(topic_emb, content_emb)
                val_losses.append(loss.item() * len(labels))
                topic_embs.append(topic_emb.detach().cpu().numpy())
                content_embs.append(content_emb.detach().cpu().numpy())
                val_nums.append(len(labels))

    val_loss = sum(val_losses) / sum(val_nums)
    topic_embs = np.concatenate(topic_embs)
    content_embs = np.concatenate(content_embs)
    metrics_dict = compute_metrics(topic_embs, content_embs, valid_df, k=[5, 10, 50])
    val_score = metrics_dict["recall@50"]
    LOGGER.info(f"Fold: {fold}, Epoch: {epoch}/{_cfg.n_epoch}, Step: {step} | val_loss: {np.round(val_loss, 5)}, \n metrics: {metrics_dict}")


    if val_score > best_val_score:
        best_val_score = val_score
        torch.save(
            model.state_dict(),
            os.path.join(_cfg.EXP_MODEL, f"cand_fold{fold}.pth")
        )
        pickle.dump(topic_emb, open(pathlib.Path(_cfg.OUTPUT_EXP) / f"topic_emb_fold{fold}.pkl", "wb"))
        pickle.dump(content_emb, open(pathlib.Path(_cfg.OUTPUT_EXP) / f"content_emb_fold{fold}.pkl", "wb"))
        es_count = 0
    elif es_count == _cfg.es_patience:
        es_count += 100
    else:
        es_count += 1
    
    LOGGER.info(f"Early stopping count: {es_count}")

    return best_val_score, val_score, es_count



def get_cand_train_loop(_cfg, train_data: pd.DataFrame, cv_list: List, correlation_df: pd.DataFrame):
    """_summary_

    Args:
        _cfg (_type_): _description_
        train_data (pd.DataFrame): textとtargetが格納されたデータフレーム
        cv_list (List): _description_
        correlation_df (pd.DataFrame): _description_

    Returns:
        _type_: _description_
    """

    train_data["fold"] = -1
    for fold in _cfg.train_fold:
        LOGGER.info(f'{"="*30} Fold{fold} {"="*30}')

        train_idx, valid_idx = cv_list[fold]
        train_data.loc[valid_idx, "fold"] = fold
        train_df = train_data[train_data["fold"]!=fold].reset_index(drop=True)
        valid_df = train_data[train_data["fold"]==fold].reset_index(drop=True)

        # Datasetの設定
        train_dataset = MetricDataset(_cfg, train_df)
        valid_dataset = MetricDataset(_cfg, valid_df)
        train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=_cfg.batch_size,
            shuffle=True,
            pin_memory=True,
            drop_last=True,
        )
        valid_loader = DataLoader(
            dataset=valid_dataset,
            batch_size=_cfg.batch_size * 2,
            shuffle=False,
            pin_memory=True,
            drop_last=False,
        )

        # model
        model = MetricModel(_cfg)
        model.config.save_pretrained(pathlib.Path(cfg.EXP_MODEL)/"1st_model_config")
        model = model.to(_cfg.device)

        criterion = MultipleNegativesRankingLoss()

        # optimizer, scheduler
        optimizer_grouped_parameters = get_optimizer_grouped_parameters(_cfg, model)
        optimizer = AdamW(optimizer_grouped_parameters, lr=_cfg.encoder_lr, eps=_cfg.eps, betas=_cfg.betas, weight_decay=_cfg.weight_decay)

        num_train_steps = int(len(train_df) / _cfg.batch_size * _cfg.n_epoch)
        scheduler = get_scheduler(_cfg, optimizer, num_train_steps)


        # model-training
        best_val_score = -np.inf
        es_count = 0

        for epoch in range(_cfg.n_epoch):
            train_loss, es_count = get_cand_train_fn(
                _cfg,
                train_loader,
                valid_loader,
                train_df,
                valid_df,
                criterion,
                optimizer,
                scheduler,
                model,
                fold,
                epoch,
                best_val_score,
                es_count,
                )
            best_val_score, val_score, es_count = get_cand_valid_fn(
                _cfg,
                valid_loader,
                valid_df,
                criterion,
                model,
                fold,
                epoch,
                "fold",
                best_val_score,
                es_count,
                )

            if _cfg.wandb:
                wandb.log({
                    f"[fold{fold}] epoch": epoch,
                    f"[fold{fold}] avg_train_loss": train_loss,
                    f"[fold{fold}] avg_val_score": val_score,
                    # f"[fold{fold}] score": best_val_score
                    })
            
            if es_count >= 100:
                break
        del model
        gc.collect()
        torch.cuda.empty_cache()

In [15]:
def get_feature_df(train_text_df, content_df, topic_df, content_emb, topic_emb):

    # content features
    content_svd_cols = [
        "content_emb",
    ]
    content_cat_cols = [
        "content_kind",
        # "content_language",
    ]

    # topic features
    topic_svd_cols = [
        "topic_emb",
    ]
    topic_cat_cols = [
        # "topic_language",
    ]

    content_blocks = [
        LabelEncodingBlock(cols=content_cat_cols, cfg=cfg, path=pathlib.Path(cfg.OUTPUT_EXP)/"content_oe.pkl"), 
        SVDBlock(cols=content_svd_cols, cfg=cfg, dim=32, title_vec=content_emb, path=pathlib.Path(cfg.OUTPUT_EXP)/"svd.pkl"),
        ]
    topic_blocks = [
        IdentityBlock(cols=["topic_level"]),
        # LabelEncodingBlock(cols=topic_cat_cols, cfg=cfg, path=pathlib.Path(cfg.OUTPUT_EXP)/"topic_oe.pkl"), 
        SVDBlock(cols=topic_svd_cols, cfg=cfg, dim=32, title_vec=topic_emb, path=pathlib.Path(cfg.OUTPUT_EXP)/"topic_svd.pkl"),
        ]
    content_feat_df = run_blocks(content_df, blocks=content_blocks, is_test=False)
    topic_feat_df = run_blocks(topic_df, blocks=topic_blocks, is_test=False)

    # content_idとfeatを対応付ける
    content_feat_df = pd.concat([content_df[["content_id"]], content_feat_df], axis=1)
    # topic_idとfeatを対応付ける
    topic_feat_df = pd.concat([topic_df[["topic_id"]], topic_feat_df], axis=1)

    # topicとcontentのfeatをマージ
    train_feat_df = pd.merge(train_text_df[["topic_id", "content_id"]], content_feat_df, on="content_id", how="left")
    train_feat_df = pd.merge(train_feat_df, topic_feat_df, on="topic_id", how="left")
    train_feat_df = train_feat_df.drop(columns=["topic_id", "content_id"])

    target_df = train_text_df["target"]
    display(train_feat_df.head())
    LOGGER.info(f"n_features: {len(train_feat_df.columns)}")

    return train_feat_df, target_df

# 2nd Stage Functions

In [16]:
def create_emb_feature_df(input_df):
    input_df["text"] = input_df["topic_title"] + " [SEP] " \
        + input_df["topic_description"] + " [SEP] " \
        + input_df["topic_context"] + " [SEP] " \
        + input_df["content_title"] + " [SEP] " \
        + input_df["content_description"] + " [SEP] " \
        + input_df["content_kind"] + " [SEP] " \
        + input_df["content_text"]
    return input_df[[
        "topic_id", 
        "content_id", 
        "topic_category", 
        "topic_channel", 
        "topic_level",
        "content_kind",
        "text", 
        "target"
        ]]

In [17]:
def get_cand_df(topic_df, content_df, topic_emb, content_emb, n_neighbors=10):
    """
    languageごとに各topicにn_neighbors個のcontentを割り当てる
    """

    topic_pred_df = []
    for _lang in tqdm(topic_df["topic_language"].unique()):
        # 対象のlanguageのindex
        content_lang_idx = content_df["content_language"] == _lang
        topic_lang_idx = topic_df["topic_language"] == _lang

        # 対象languageのdataframe
        content_lang_df = content_df[content_lang_idx].reset_index()
        topic_lang_df = topic_df[topic_lang_idx].reset_index()

        # 対象languageのembeddings
        content_lang_vec = content_emb[content_lang_idx]
        topic_lang_vec = topic_emb[topic_lang_idx]

        content_lang_vec_gpu = cp.array(content_lang_vec)
        topic_lang_vec_gpu = cp.array(topic_lang_vec)
        neighbors_model = NearestNeighbors(n_neighbors=n_neighbors, metric="cosine")
        neighbors_model.fit(content_lang_vec_gpu)
        indices = neighbors_model.kneighbors(topic_lang_vec_gpu, return_distance=False)

        preds = []
        for _content_indices in range(len(indices)):
            idx_pred = indices[_content_indices]
            str_pred = " ".join([content_lang_df.loc[content_idx, "content_id"] for content_idx in idx_pred.get()])
            preds.append(str_pred)
        topic_lang_pred_df = topic_lang_df.copy()
        topic_lang_pred_df["content_id"] = preds
        topic_pred_df.append(topic_lang_pred_df)

        del content_lang_vec_gpu, topic_lang_vec_gpu, neighbors_model, indices, preds, topic_lang_pred_df
        gc.collect()
        torch.cuda.empty_cache()

    topic_pred_df = pd.concat(topic_pred_df).sort_values("index").drop(columns="index").reset_index(drop=True)

    return topic_pred_df[["topic_id", "content_id"]]

In [18]:
class EmbDataset(Dataset):
    def __init__(self, cfg, df, col):
        self.cfg = cfg
        self.text = df[col].to_numpy()

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.prepare_input(self.cfg, self.text[index])
        return text

    @staticmethod
    def prepare_input(cfg, text):
        inputs = cfg.tokenizer(
            text,
            add_special_tokens=True,
            max_length=cfg.max_len,
            padding="max_length",
            truncation=True,
            return_offsets_mapping=False
            )
        inputs['input_ids'] = torch.tensor(
            inputs['input_ids'],
            dtype=torch.long
        )
        inputs['attention_mask'] = torch.tensor(
            inputs['attention_mask'],
            dtype=torch.long
        )
        inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
        }
        return inputs

In [19]:
def get_cand_inference_fn(_cfg, data_loader, model):
    embs = []
    model.eval()
    with torch.no_grad():
        with tqdm(data_loader, total=len(data_loader)) as pbar:
            for inputs in pbar:
                inputs = collate(inputs)
                for k, v in inputs.items():
                    inputs[k] = v.to(_cfg.device)
                with autocast(enabled=_cfg.apex):
                    emb = model.forward_once(inputs)
                emb = emb.detach().cpu().numpy()
                embs.append(emb)
    embs = np.concatenate(embs)
    return embs

def get_cand_fn(_cfg, topic_df, content_df):
    topic_dataset = EmbDataset(_cfg, topic_df, "topic_text")
    content_dataset = EmbDataset(_cfg, content_df, "content_text")
    topic_loader = DataLoader(
        dataset=topic_dataset,
        batch_size=_cfg.batch_size*2,
        shuffle=False,
        pin_memory=True,
        drop_last=False
    )
    content_loader = DataLoader(
        dataset=content_dataset,
        batch_size=_cfg.batch_size*2,
        shuffle=False,
        pin_memory=True,
        drop_last=False
    )
    
    model = MetricModel(_cfg)
    model.load_state_dict(torch.load(f"/home/working/output/exp025/model/cand_fold0.pth"))
    model = model.to(_cfg.device)
    topic_emb = get_cand_inference_fn(_cfg, topic_loader, model)
    content_emb = get_cand_inference_fn(_cfg, content_loader, model)


    del model
    gc.collect()
    torch.cuda.empty_cache()

    return topic_emb, content_emb

In [20]:
# =====================
# Dataset, Model
# =====================

def processing_features(df):
    df['text'] = df['text'].apply(lambda x : resolve_encodings_and_normalize(x))
    return df

class BiEncoderDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.text = df["text"].to_numpy()
        self.label = df["target"].to_numpy()

    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):
        text = self.prepare_input(self.cfg, self.text[index])
        label = self.label[index].astype(np.float32)
        return text, label

    @staticmethod
    def prepare_input(cfg, text):
        inputs = cfg.tokenizer(
            text,
            add_special_tokens=True,
            max_length=cfg.max_len,
            padding="max_length",
            truncation=True,
            return_offsets_mapping=False
            )
        inputs['input_ids'] = torch.tensor(
            inputs['input_ids'],
            dtype=torch.long
        )
        inputs['attention_mask'] = torch.tensor(
            inputs['attention_mask'],
            dtype=torch.long
        )
        inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
        }
        return inputs


In [21]:
class CustomModel(nn.Module):
    def __init__(self, cfg): 
        super().__init__()
        self.cfg = cfg
        self.gpu_optimize_config = cfg.gpu_optimize_config
        self.config = AutoConfig.from_pretrained(
            cfg.MODEL_PATH,
            output_hidden_states=True
        )
        self.config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout": 0.,
                "hidden_dropout_prob": 0.,
                "attention_dropout": 0.,
                "attention_probs_dropout_prob": 0,
            }
        )
        self.model = AutoModel.from_pretrained(
            cfg.MODEL_PATH,
            config=self.config
        )
        self.model.resize_token_embeddings(len(self.cfg.tokenizer))

        # self.pool = AttentionPooling(self.config.hidden_size)
        # self.weighted_layer_pool = WeightedLayerPooling(self.config.num_hidden_layers)
        # self.pool = MeanPooling()
        # self.fc = nn.Linear(self.config.hidden_size, cfg.num_classes)
        # self._init_weights(self.fc)
        self.ln = nn.LayerNorm(self.config.hidden_size)
        self._init_weights(self.ln)

        self.drop1 = nn.Dropout(0.1)
        self.drop2 = nn.Dropout(0.2)
        self.drop3 = nn.Dropout(0.3)
        self.drop4 = nn.Dropout(0.4)
        self.drop5 = nn.Dropout(0.5)

        # Freeze
        if self.gpu_optimize_config['freezing']:
            freeze(self.model.encoder.layer[:4])

        # Gradient Checkpointing
        if self.gpu_optimize_config['gradient_checkpoint']:
            self.model.gradient_checkpointing_enable()

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_state = outputs[0]
        feature = self.pool(last_state, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        # batch, hidden_size
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output.squeeze()



In [22]:
def train_fn(cfg, train_loader, valid_loader, train_df, valid_df, criterion, optimizer, scheduler, model, fold, epoch, best_val_preds, best_val_score, es_count):
    LOGGER.info(f'{"="*20} epoch{epoch} {"="*20}')
    train_losses = []
    train_nums = []
    model.train()
    scaler = GradScaler(enabled=cfg.apex)
    with tqdm(train_loader, total=len(train_loader)) as pbar:
        for step, (inputs, labels) in enumerate(pbar):
            inputs = collate(inputs)
            for k, v in inputs.items():
                inputs[k] = v.to(cfg.device)
            labels = labels.unsqueeze(dim=1).to(cfg.device)
            with autocast(enabled=cfg.apex):
                output = model.forward_classification(inputs)
            loss = criterion(output, labels)

            pbar.set_postfix({
                'loss': loss.item(),
                'lr': scheduler.get_lr()[0]
            })
            train_losses.append(loss.item() * len(labels))
            train_nums.append(len(labels))

            if cfg.gradient_accumulation_steps > 1:
                loss = loss / cfg.gradient_accumulation_steps

            scaler.scale(loss).backward()

            if cfg.clip_grad_norm is not None:
                # scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(
                    model.parameters(),
                    cfg.clip_grad_norm
                )

            if (step+1) % cfg.gradient_accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            if step % cfg.eval_step == 0 and step != 0:
                best_val_preds, best_val_score, val_loss, es_count = valid_fn(
                    cfg,
                    valid_loader,
                    valid_df,
                    criterion,
                    model,
                    fold,
                    epoch,
                    step,
                    best_val_preds,
                    best_val_score,
                    es_count,
                )
                model.train()

            if cfg.wandb:
                wandb.log({f"[fold{fold}] train_loss": loss.item(),
                        f"[fold{fold}] lr": scheduler.get_lr()[0]})
    train_loss = sum(train_losses)/sum(train_nums)
    return train_loss



def valid_fn(cfg, valid_loader, valid_df, criterion, model, fold, epoch, step, best_val_preds, best_val_score, es_count):
    val_preds = []
    val_losses = []
    val_nums = []
    model.eval()
    with torch.no_grad():
        with tqdm(valid_loader, total=len(valid_loader)) as pbar:
            for (inputs, labels) in pbar:
                inputs = collate(inputs)
                for k, v in inputs.items():
                    inputs[k] = v.to(cfg.device)
                labels = labels.unsqueeze(dim=1).to(cfg.device)
                with autocast(enabled=cfg.apex):
                    output = model.forward_classification(inputs)
                loss = criterion(output, labels)
                output = output.detach().cpu().numpy()
                output = sigmoid(output)
                val_preds.append(output)
                val_losses.append(loss.item() * len(labels))
                val_nums.append(len(labels))

    val_preds = np.concatenate(val_preds)
    val_loss = sum(val_losses) / sum(val_nums)
    y_preds = np.where(val_preds>=0.05, 1, 0)
    score = fbeta_score(valid_df["target"], y_preds, beta=2)
    recall = recall_score(valid_df["target"], y_preds)

    LOGGER.info(f'Fold: {fold}, Epoch: {epoch}/{cfg.n_epoch}, Step: {step} | val_loss: {np.round(val_loss, 5)}, f2: {np.round(score, 5)}, recall: {np.round(recall, 5)}')

    if score > best_val_score:
        best_val_preds = val_preds
        best_val_score = score
        torch.save(
            model.state_dict(),
            os.path.join(cfg.EXP_MODEL, f"fold{fold}.pth")
        )
        pickle.dump(best_val_preds, open(pathlib.Path(cfg.OUTPUT_EXP) / "best_val_preds.pkl", "wb"))
        es_count = 0
    elif es_count == cfg.es_patience:
        es_count += 100
    else:
        es_count += 1
    LOGGER.info(f"Early stopping count: {es_count}")

    return best_val_preds, best_val_score, val_loss, es_count


def train_loop(cfg, train_data: pd.DataFrame):
    """_summary_

    Args:
        cfg (_type_): _description_
        train_data (pd.DataFrame): textとtargetが格納されたデータフレーム
        cv_list (List): _description_
        correlation_df (pd.DataFrame): _description_

    Returns:
        _type_: _description_
    """
    oof_pred = np.zeros((len(train_data)), dtype=np.float32)
    fold_score = []

    # train_data["fold"] = -1
    for fold in [0]:
        LOGGER.info(f'{"="*30} Fold{fold} {"="*30}')

        train_df = train_data[train_data["fold"]!=fold].reset_index(drop=True)
        valid_df = train_data[train_data["fold"]==fold]
        valid_idx = valid_df.index
        valid_df = valid_df.reset_index(drop=True)


        # Datasetの設定
        train_dataset = BiEncoderDataset(cfg, train_df)
        valid_dataset = BiEncoderDataset(cfg, valid_df)
        train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=cfg.batch_size,
            shuffle=True,
            pin_memory=True,
            drop_last=True,
        )
        valid_loader = DataLoader(
            dataset=valid_dataset,
            batch_size=cfg.batch_size,
            shuffle=False,
            pin_memory=True,
            drop_last=False,
        )

        # model
        model = MetricModel(cfg)
        model.load_state_dict(torch.load(f"/home/working/output/exp025/model/cand_fold0.pth"))
        # for param in model.parameters():
        #     param.requires_grad = False  # 勾配を計算しない
        # model.pool = MeanPooling()
        # model.fc = nn.Linear(model.config.hidden_size, cfg.num_classes)
        # model._init_weights(model.fc)
        
        model.config.save_pretrained(pathlib.Path(cfg.EXP_MODEL)/"2nd_model_config")
        model = model.to(cfg.device)
        
        criterion = nn.BCEWithLogitsLoss()

        # optimizer, scheduler
        optimizer_grouped_parameters = get_optimizer_grouped_parameters(cfg, model)
        optimizer = AdamW(optimizer_grouped_parameters, lr=cfg.encoder_lr, eps=cfg.eps, betas=cfg.betas, weight_decay=cfg.weight_decay)

        num_train_steps = int(len(train_df) / cfg.batch_size * cfg.n_epoch)
        scheduler = get_scheduler(cfg, optimizer, num_train_steps)


        # model-training
        best_val_preds = None
        best_val_score = -np.inf
        es_count = 0

        for epoch in range(cfg.n_epoch):
            train_loss = train_fn(
                cfg,
                train_loader,
                valid_loader,
                train_df,
                valid_df,
                criterion,
                optimizer,
                scheduler,
                model,
                fold,
                epoch,
                best_val_preds,
                best_val_score,
                es_count,
                )

            LOGGER.info(f'Fold{fold}, Epoch{epoch}/{cfg.n_epoch} | train_loss: {np.round(train_loss, 5)}')
            best_val_preds, best_val_score, val_loss, es_count = valid_fn(
                cfg,
                valid_loader,
                valid_df,
                criterion,
                model,
                fold,
                epoch,
                'end',
                best_val_preds,
                best_val_score,
                es_count,
            )


            if cfg.wandb:
                wandb.log({f"[fold{fold}] epoch": epoch,
                        f"[fold{fold}] avg_train_loss": train_loss,
                        f"[fold{fold}] avg_val_loss": val_loss,
                        f"[fold{fold}] score": best_val_score})

            if es_count >= 100:
                break

        oof_pred[valid_idx] = best_val_preds.squeeze().astype(np.float32)
        np.save(os.path.join(cfg.EXP_PREDS, f'oof_pred_fold{fold}.npy'), best_val_preds)
        fold_score.append(best_val_score)
        del model
        gc.collect()
        torch.cuda.empty_cache()
        break

    np.save(os.path.join(cfg.EXP_PREDS, 'oof_pred.npy'), oof_pred)

    # =====================
    # scoring
    # =====================
    # score = calc_comp_score(train_data, oof_pred, correlation_df)
    # LOGGER.info(f'fold score: {fold_score}')
    # LOGGER.info(f'CV: {round(score, 4)}')
    # return score

# Setup & Preprocessing

In [23]:
content_df, topic_df, correlation_df, sub_df = get_processed_df()

In [24]:
# get context
is_overwrite = False
filepath = pathlib.Path(cfg.OUTPUT_EXP) / "topic_context.csv"
if filepath.exists() and not is_overwrite:
    topic_context = pd.read_csv(filepath)
else:
    topic_context = topic_df["topic_id"].parallel_apply(get_topic_context)
    topic_context.to_csv(filepath, index=False)
topic_df["topic_context"] = topic_context

In [25]:
# remove topic has no content
topic_df = topic_df[topic_df["topic_has_content"]].reset_index(drop=True)

topic_emb_feature_df = create_topic_emb_feature_df(topic_df)
content_emb_feature_df = create_content_emb_feature_df(content_df)

# correlation_dfを広げる
correlation_df_ = correlation_df.copy()
correlation_df_["content_id"] = correlation_df["content_id"].apply(lambda x: x.split(" "))
correlation_df_ = correlation_df_.explode("content_id")

# correlation_dfに含まれるtopicとcontentの組み合わせを学習データとする
cand_df = pd.merge(correlation_df_, topic_emb_feature_df, on="topic_id", how="left")
cand_df = pd.merge(cand_df, content_emb_feature_df, on="content_id", how="left")
cand_df["target"] = 1

cand_df

,topic_id,content_id,topic_channel,topic_category,topic_text,content_text,target
0,t_00004da3a1b2,c_1108dd0c7a5d,000cf7,source,Откриването на резисторите [SEP] Изследване на...,Молив като резистор [SEP] Моливът причинява пр...,1
1,t_00004da3a1b2,c_376c5a8eb028,000cf7,source,Откриването на резисторите [SEP] Изследване на...,Да чуем променливото съпротивление [SEP] Тук ч...,1
2,t_00004da3a1b2,c_5bc0e1e2cba0,000cf7,source,Откриването на резисторите [SEP] Изследване на...,Променлив резистор (реостат) с графит от молив...,1
3,t_00004da3a1b2,c_76231f9d0b5e,000cf7,source,Откриването на резисторите [SEP] Изследване на...,Последователно свързване на галваничен елемент...,1
4,t_00068291e9a4,c_639ea2ef9c95,8e286a,source,Entradas e saídas de uma função [SEP] Entenda ...,Dados e resultados de funções: gráficos [SEP] ...,1
...,...,...,...,...,...,...,...
279914,t_fff9e5407d13,c_d64037a72376,71fd51,supplemental,NA_U06 - El periódico [SEP] [SEP] PF (Español...,Introducción: El periódico [SEP] [SEP] html5 ...,1
279915,t_fffbe1d5d43c,c_46f852a49c08,0c929f,source,Inscribed shapes problem solving [SEP] Use pro...,Proof: Right triangles inscribed in circles -d...,1
279916,t_fffbe1d5d43c,c_6659207b25d5,0c929f,source,Inscribed shapes problem solving [SEP] Use pro...,Area of inscribed equilateral triangle -dubbed...,1
279917,t_fffe14f1be1e,c_cece166bad6a,6e90a7,aligned,Lección 7 [SEP] [SEP] CREE Para el Estudiante...,Juego con las palabras [SEP] [SEP] document [...,1


# 1st Stage

In [26]:
metric_cfg.tokenizer = AutoTokenizer.from_pretrained(metric_cfg.MODEL_PATH)
# special_tokens_dict = {'additional_special_tokens': ["[SEP_FEATURE]"]}
# metric_cfg.tokenizer.add_special_tokens(special_tokens_dict)
metric_cfg.tokenizer.save_pretrained(pathlib.Path(metric_cfg.OUTPUT_EXP) / "1st_tokenizer")

('/home/working/output/exp025/1st_tokenizer/tokenizer_config.json',
 '/home/working/output/exp025/1st_tokenizer/special_tokens_map.json',
 '/home/working/output/exp025/1st_tokenizer/vocab.txt',
 '/home/working/output/exp025/1st_tokenizer/added_tokens.json',
 '/home/working/output/exp025/1st_tokenizer/tokenizer.json')

In [27]:
cand_cv_list = get_GroupKFold_list(X=cand_df, y=cand_df["target"], groups=cand_df["topic_channel"], n_splits=metric_cfg.num_fold, seed=metric_cfg.seed)

# fold0
fold0_topic_id = cand_df.iloc[cand_cv_list[0][1]]["topic_id"].unique()

get_cand_train_loop(metric_cfg, cand_df, cand_cv_list, correlation_df)

============================== Fold0 ==============================
==================== epoch0 ====================


  0%|          | 0/218 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

Fold: 0, Epoch: 0/30, Step: 150 | val_loss: 4.16779, 
 metrics: {'recall@5': 0.32981, 'f2@5': 0.29457, 'recall@10': 0.41224, 'f2@10': 0.31262, 'recall@50': 0.62087, 'f2@50': 0.25294}
Early stopping count: 0


In [ ]:
is_overwrite = True
filepath = pathlib.Path(cfg.OUTPUT_EXP) / "topic_emb.pkl"
if not filepath.exists() or is_overwrite:
    topic_emb, content_emb = get_cand_fn(metric_cfg, topic_emb_feature_df, content_emb_feature_df)
    pickle.dump(topic_emb, open(pathlib.Path(cfg.OUTPUT_EXP) / "topic_emb.pkl", "wb"))
    pickle.dump(content_emb, open(pathlib.Path(cfg.OUTPUT_EXP) / "content_emb.pkl", "wb"))
    train_text_df = get_cand_df(topic_df, content_df, topic_emb, content_emb, n_neighbors=50)
    train_text_df.to_csv(f"{cfg.OUTPUT_EXP}/train_text_df.csv", index=False)
else:
    topic_emb = pickle.load(open(pathlib.Path(cfg.OUTPUT_EXP) / "topic_emb.pkl", "rb"))
    content_emb = pickle.load(open(pathlib.Path(cfg.OUTPUT_EXP) / "content_emb.pkl", "rb"))
    train_text_df = pd.read_csv(f"{cfg.OUTPUT_EXP}/train_text_df.csv")

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
_correlation_df = correlation_df[correlation_df["topic_id"].isin(fold0_topic_id)]
_train_text_df = train_text_df[train_text_df["topic_id"].isin(fold0_topic_id)]

cv_score = comp_recall_score(_correlation_df["content_id"], _train_text_df["content_id"])

print(np.round(cv_score, 5))

0.76389


In [ ]:
train_text_df_6 = get_cand_df(topic_df, content_df, topic_emb, content_emb, n_neighbors=10)
_correlation_df = correlation_df[correlation_df["topic_id"].isin(fold0_topic_id)]
_train_text_df = train_text_df_6[train_text_df_6["topic_id"].isin(fold0_topic_id)]
pred_df_ = pd.merge(_correlation_df[["topic_id"]], _train_text_df, on="topic_id", how="left")
pred_df_ = pred_df_.fillna("")

cv_score = comp_recall_score(_correlation_df["content_id"], pred_df_["content_id"])
cv_score1 = comp_fbeta_score(_correlation_df["content_id"], pred_df_["content_id"])

# del pred_df_
gc.collect()

print(np.round(cv_score, 5))
print(np.round(cv_score1, 5))

  0%|          | 0/27 [00:00<?, ?it/s]

0.57055
0.40107


# 2nd Stage

In [ ]:
train_text_df["content_id"] = train_text_df["content_id"].apply(lambda x: x.split(" "))
train_text_df = train_text_df.explode("content_id").reset_index(drop=True)

train_text_df = pd.merge(train_text_df, cand_df[["topic_id", "content_id"]], on=["topic_id", "content_id"], how="outer")

train_text_df = pd.merge(train_text_df, topic_df, on="topic_id", how="left")
train_text_df = pd.merge(train_text_df, content_df, on="content_id", how="left")

In [ ]:
# create target
_correlation_df = correlation_df.copy()
_correlation_df["content_id"] = _correlation_df["content_id"].apply(lambda x: x.split(" "))
_correlation_df = _correlation_df.explode("content_id")
_correlation_df["target"] = 1
train_text_df = pd.merge(train_text_df, _correlation_df, on=["topic_id", "content_id"], how="outer")
train_text_df = create_emb_feature_df(train_text_df)
train_text_df["target"] = train_text_df["target"].fillna(0) # correlation_dfにないペアは0とする

# create fold
train_text_df["fold"] = -1
train_text_df.loc[train_text_df["topic_id"].isin(fold0_topic_id), "fold"] = 0
train_text_df

,topic_id,content_id,topic_category,topic_channel,topic_language,topic_level,content_language,content_kind,text,target,fold
0,t_00004da3a1b2,c_1108dd0c7a5d,source,000cf7,bg,4,bg,video,Откриването на резисторите [SEP_FEATURE] Изсле...,1.0,-1
1,t_00004da3a1b2,c_5bc0e1e2cba0,source,000cf7,bg,4,bg,video,Откриването на резисторите [SEP_FEATURE] Изсле...,1.0,-1
2,t_00004da3a1b2,c_8fae27abecfb,source,000cf7,bg,4,bg,video,Откриването на резисторите [SEP_FEATURE] Изсле...,0.0,-1
3,t_00004da3a1b2,c_e1e8557d7c61,source,000cf7,bg,4,bg,video,Откриването на резисторите [SEP_FEATURE] Изсле...,0.0,-1
4,t_00004da3a1b2,c_0feaaa5dc39d,source,000cf7,bg,4,bg,video,Откриването на резисторите [SEP_FEATURE] Изсле...,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
3128341,t_fff1047917af,c_f59987cf8a75,source,217993,pnb,4,pnb,video,ਅਧਿਆਪਕਾਂ ਲਈ [SEP_FEATURE] source_id=LES1229 [S...,1.0,-1
3128342,t_fff5da49c4d3,c_cbceabf1d5d6,source,735876,en,3,en,html5,Business Writing [SEP_FEATURE] [SEP_FEATURE] ...,1.0,-1
3128343,t_fff9e5407d13,c_4edb6a46f75e,supplemental,71fd51,es,2,es,html5,NA_U06 - El periódico [SEP_FEATURE] [SEP_FEAT...,1.0,0
3128344,t_fff9e5407d13,c_90cc64d55d46,supplemental,71fd51,es,2,es,html5,NA_U06 - El periódico [SEP_FEATURE] [SEP_FEAT...,1.0,0


In [ ]:
# train_feat_df, target_df = get_feature_df(
#     train_text_df,
#     content_df,
#     topic_df,
#     content_emb,
#     topic_emb,
#     )

******************** start run blocks... ********************
	- <kagglib.tabular.blocks.LabelEncodingBlock object at 0x7f25e8c000d0> 0.018[s]
	- <kagglib.tabular.blocks.SVDBlock object at 0x7f25e8c007d0> 3.296[s]
run is_test=False 3.333[s]
******************** start run blocks... ********************
	- <kagglib.tabular.blocks.IdentityBlock object at 0x7f25e8c00850> 0.000[s]
	- <kagglib.tabular.blocks.SVDBlock object at 0x7f25e8c002d0> 1.439[s]
run is_test=False 1.453[s]


,content_kind@LabelEncodingBlock,content_emb_0@SVDBlock,content_emb_1@SVDBlock,content_emb_2@SVDBlock,content_emb_3@SVDBlock,content_emb_4@SVDBlock,content_emb_5@SVDBlock,content_emb_6@SVDBlock,content_emb_7@SVDBlock,content_emb_8@SVDBlock,content_emb_9@SVDBlock,content_emb_10@SVDBlock,content_emb_11@SVDBlock,content_emb_12@SVDBlock,content_emb_13@SVDBlock,content_emb_14@SVDBlock,content_emb_15@SVDBlock,content_emb_16@SVDBlock,content_emb_17@SVDBlock,content_emb_18@SVDBlock,content_emb_19@SVDBlock,content_emb_20@SVDBlock,content_emb_21@SVDBlock,content_emb_22@SVDBlock,content_emb_23@SVDBlock,content_emb_24@SVDBlock,content_emb_25@SVDBlock,content_emb_26@SVDBlock,content_emb_27@SVDBlock,content_emb_28@SVDBlock,content_emb_29@SVDBlock,content_emb_30@SVDBlock,content_emb_31@SVDBlock,topic_level@IdentityBlock,topic_emb_0@SVDBlock,topic_emb_1@SVDBlock,topic_emb_2@SVDBlock,topic_emb_3@SVDBlock,topic_emb_4@SVDBlock,topic_emb_5@SVDBlock,topic_emb_6@SVDBlock,topic_emb_7@SVDBlock,topic_emb_8@SVDBlock,topic_emb_9@SVDBlock,topic_emb_10@SVDBlock,topic_emb_11@SVDBlock,topic_emb_12@SVDBlock,topic_emb_13@SVDBlock,topic_emb_14@SVDBlock,topic_emb_15@SVDBlock,topic_emb_16@SVDBlock,topic_emb_17@SVDBlock,topic_emb_18@SVDBlock,topic_emb_19@SVDBlock,topic_emb_20@SVDBlock,topic_emb_21@SVDBlock,topic_emb_22@SVDBlock,topic_emb_23@SVDBlock,topic_emb_24@SVDBlock,topic_emb_25@SVDBlock,topic_emb_26@SVDBlock,topic_emb_27@SVDBlock,topic_emb_28@SVDBlock,topic_emb_29@SVDBlock,topic_emb_30@SVDBlock,topic_emb_31@SVDBlock
0,4.0,0.079212,0.125695,-0.469802,-0.494434,0.138742,0.633188,0.144982,0.413376,0.169338,-0.323654,-0.399534,0.165209,-0.131193,0.678805,-0.078516,-0.216137,0.392778,-0.157129,0.882506,0.129295,0.138295,-0.267511,0.006710,0.049550,0.061713,0.298734,-0.113854,0.338589,0.340593,-0.157364,-0.183485,0.192407,4,-0.040972,0.101971,0.662097,0.08142,-0.175815,-0.057184,-0.230083,0.198315,0.248321,0.13243,0.58337,0.721311,-0.133335,-0.348155,0.144732,-0.085495,-0.08335,0.475384,0.608218,-0.108132,0.021382,0.236513,0.161072,0.041059,0.599249,0.092332,-0.20305,0.126148,0.047797,-0.060872,0.237958,0.029725
1,4.0,-0.037359,0.048075,-0.283860,-0.384652,0.109580,0.306531,0.331767,0.415248,0.109764,0.044823,-0.254070,0.584265,-0.144821,0.817602,-0.119610,-0.283553,0.340146,-0.194227,0.583025,0.105869,-0.194760,-0.225125,-0.406466,0.121462,0.551967,0.083317,0.032296,-0.031025,0.023685,0.333466,-0.150630,0.107592,4,-0.040972,0.101971,0.662097,0.08142,-0.175815,-0.057184,-0.230083,0.198315,0.248321,0.13243,0.58337,0.721311,-0.133335,-0.348155,0.144732,-0.085495,-0.08335,0.475384,0.608218,-0.108132,0.021382,0.236513,0.161072,0.041059,0.599249,0.092332,-0.20305,0.126148,0.047797,-0.060872,0.237958,0.029725
2,4.0,0.105531,0.130914,-0.534807,-0.365405,-0.044168,0.307978,-0.046559,0.560299,0.132553,-0.205845,-0.322072,0.395685,0.025463,0.421429,-0.115012,-0.252980,0.563537,-0.115485,0.784990,0.168368,-0.101050,-0.291281,-0.113878,0.309886,0.169137,0.173713,0.117425,0.175831,0.224545,0.365263,-0.005221,0.047211,4,-0.040972,0.101971,0.662097,0.08142,-0.175815,-0.057184,-0.230083,0.198315,0.248321,0.13243,0.58337,0.721311,-0.133335,-0.348155,0.144732,-0.085495,-0.08335,0.475384,0.608218,-0.108132,0.021382,0.236513,0.161072,0.041059,0.599249,0.092332,-0.20305,0.126148,0.047797,-0.060872,0.237958,0.029725
3,4.0,0.016613,0.168699,-0.394633,-0.541992,-0.196599,0.461443,0.042263,0.484258,0.274188,-0.241251,-0.124617,0.372174,-0.140188,0.640545,0.002009,-0.069064,0.500712,-0.167379,0.714358,0.030145,0.050235,-0.208665,-0.183565,0.167504,0.104902,0.086236,-0.111964,0.349275,0.133548,0.341001,0.032715,0.062358,4,-0.040972,0.101971,0.662097,0.08142,-0.175815,-0.057184,-0.230083,0.198315,0.248321,0.13243,0.58337,0.721311,-0.133335,-0.348155,0.144732,-0.085495,-0.08335,0.475384,0.608218,-0.108132,0.021382,0.236513,0.161072,0.041059,0.599249,0.092332,-0.20305,0.126148,0.047797,-0.060872,0.237958,0.029725
4,4.0,0.143173,0.257535,-0.515924,-0.374057,-0.094327,0.439797,-0.061152,0.48515

n_features: 66


In [ ]:
def train_cv(cfg, train_data, target_df, metrics_dict, LOGGER, fold_df):
    """交差検証を実行"""
    oof = np.zeros(len(train_data))
    models = []

    for i_fold in [0]:
        LOGGER.info(decorate(f"Fold{i_fold}", decoration="=="))
        filepath = os.path.join(cfg.OUTPUT_EXP, f"{cfg.gbdt_model}_fold_{i_fold}.pkl")

        train_idx = fold_df[fold_df["fold"]!=i_fold].index
        valid_idx = fold_df[fold_df["fold"]==i_fold].index
        X_train = train_data.iloc[train_idx].to_numpy()
        X_valid = train_data.iloc[valid_idx].to_numpy()
        y_train = target_df.iloc[train_idx].to_numpy()
        y_valid = target_df.iloc[valid_idx].to_numpy()

        model = get_model(cfg, cfg.gbdt_model)
        model.fit(X_train, y_train, X_valid, y_valid)
        save_model(filepath, model)

        model = load_model(filepath)
        models.append(model)
        y_prob = model.predict(X_valid)
        y_prob = y_prob[:, 1].squeeze()
        y_preds_ = np.where(y_prob >= 1e-2, 1, 0)
        metrics_dict_scored = metrics_dict(y_valid, y_preds_)

        for key in metrics_dict_scored.keys():
            LOGGER.info(f"{key}: {np.round(metrics_dict_scored[key], 5)}")
        oof[valid_idx] = y_prob

    LOGGER.info(decorate("OOF"))
    oof_ = np.where(oof >= 1e-2, 1, 0)
    metrics_dict_scored = metrics_dict(target_df, oof_)
    for key in metrics_dict_scored.keys():
        LOGGER.info(f"Fold{i_fold} {key}: {np.round(metrics_dict_scored[key], 5)}")

    pickle.dump(oof, open(os.path.join(cfg.EXP_PREDS, "oof.pkl"), "wb"))
    return oof, models

In [ ]:
# def metrics_dict(y_true, y_pred):
#     _metrics_dict = {}
#     _metrics_dict["f2_score"] = fbeta_score(y_true, y_pred, beta=2)
#     _metrics_dict["recall"] = recall_score(y_true, y_pred)
#     return _metrics_dict

# filepath = os.path.join(cfg.EXP_PREDS, "oof_2nd.pkl")
# if os.path.isfile(filepath):
#     oof = pickle.load(open(filepath, "rb"))
# else:
#     oof, models = train_cv(cfg, train_feat_df, target_df, metrics_dict=metrics_dict, LOGGER=LOGGER, fold_df=train_text_df[["fold"]])
#     pickle.dump(oof, open(filepath, "wb"))

======================================== Fold0 ========================================


[0]	validation_0-logloss:0.52804
[500]	validation_0-logloss:0.31496
[516]	validation_0-logloss:0.31501


f2_score: 0.35592
recall: 0.98541
==================== OOF ====================
Fold0 f2_score: 0.1651
Fold0 recall: 0.19709


In [ ]:
# _correlation_df = correlation_df[correlation_df["topic_id"].isin(fold0_topic_id)]
# _train_text_df = train_text_df[train_text_df["topic_id"].isin(fold0_topic_id)]

# oof = oof[oof!=0]
# oof_preds = np.where(oof>=1e-1, 1, 0)
# reduced_pred_df = _train_text_df[oof_preds==1]
# calc_pred_df = reduced_pred_df.groupby("topic_id")["content_id"].apply(list).apply(" ".join)
# calc_pred_df = pd.merge(_correlation_df[["topic_id"]], calc_pred_df, on="topic_id", how="left")
# calc_pred_df = calc_pred_df.fillna("nan")

# reduced_recall_score = comp_recall_score(_correlation_df["content_id"], calc_pred_df["content_id"])
# LOGGER.info(f"Filtering by GBDT: \n recall = {np.round(reduced_recall_score, 5)} \n n_data = {len(reduced_pred_df)}")

Filtering by GBDT: 
 recall = 0.54425 
 n_data = 202585


In [ ]:
# oof_pred = np.where(oof>=1e-2, 1, 0)
# train_text_df = train_text_df[oof_pred==1].reset_index(drop=True)

In [ ]:
cfg.tokenizer = metric_cfg.tokenizer
cfg.tokenizer.save_pretrained(pathlib.Path(cfg.OUTPUT_EXP) / "2nd_tokenizer")

score = train_loop(cfg, train_text_df)

============================== Fold0 ==============================
==================== epoch0 ====================


  0%|          | 0/3336 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
valid_df = train_text_df[train_text_df["fold"]==0]
valid_df = valid_df[valid_df["topic_category"]!="source"]

best_val_preds = pickle.load(open(pathlib.Path(cfg.OUTPUT_EXP) / "best_val_preds.pkl", "rb"))

In [ ]:
for i in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12]:
    y_preds = np.where(best_val_preds>=i, 1, 0)
    score = fbeta_score(valid_df["target"], y_preds, beta=2)
    recall = recall_score(valid_df["target"], y_preds)
    print(f"{i}:", np.round(score, 5), np.round(recall, 5))

0.01: 0.50413 1.0
0.02: 0.50413 1.0
0.03: 0.50414 1.0
0.04: 0.50444 0.99971
0.05: 0.50541 0.99864
0.06: 0.50655 0.99664
0.07: 0.50708 0.99318
0.08: 0.5066 0.98791
0.09: 0.50622 0.98309
0.1: 0.50518 0.97695
0.11: 0.50383 0.96969
0.12: 0.50188 0.9615


In [38]:
# if cfg.upload_from_colab:
#     dataset_create_new(dataset_name=cfg.EXP, upload_dir=cfg.OUTPUT_EXP)

In [39]:
valid_df["pred_prob"] = best_val_preds
valid_df["pred"] = np.where(best_val_preds>=0.06, 1, 0)
valid_df = valid_df.sort_values("pred_prob", ascending=False)

In [40]:
pos = valid_df[valid_df["pred"]==1].groupby("topic_id")["content_id"].agg(list).reset_index()

# 予測contentがなければ、最も予測確率が高いcontentを割り当て
no_pos = valid_df.groupby("topic_id").head(1)
no_pos = no_pos[no_pos["pred"]==0].groupby("topic_id")["content_id"].agg(list).reset_index()

In [41]:
pred_df = pd.concat([pos, no_pos]).reset_index(drop=True)
pred_df["content_id"] = pred_df["content_id"].apply(lambda x: " ".join(x))
pred_df.columns = ["topic_id", "content_id"]
pred_df

,topic_id,content_id
0,t_0008a1bd84ba,c_8790b074383e c_bb609fdf4a75 c_cc0905ab8f4a c...
1,t_001bd01717d7,c_c6941dfa5dfc c_6f0a3cf19895 c_16f1cff519b3 c...
2,t_0020bde404c1,c_9cf7f631f662
3,t_003bb2871126,c_0c50b5b9085e c_f932eb08bb00 c_f2d57f8040de c...
4,t_0043301d4771,c_ff1fd9de6321 c_d8f8c187f8dc c_b989754fe5f4 c...
...,...,...
4526,t_ffc71a181765,c_ea92fdd9899a c_d7be46af5e2b
4527,t_ffcb44cf3348,c_de4a12d54412 c_c4308e528ecb
4528,t_ffd908252a7d,c_d84eedcb7d52 c_5d014c6f7def
4529,t_fff7782561f4,c_bca8280a9ad1 c_69b61f90d63e c_9019aa0ba185 c...


In [42]:
target_df = correlation_df[correlation_df["topic_id"].isin(valid_df["topic_id"])]
target_df

,topic_id,content_id
5,t_0008a1bd84ba,c_7ff92a954a3d c_8790b074383e
19,t_001bd01717d7,c_16f1cff519b3 c_289a31069ea7 c_6f0a3cf19895 c...
24,t_0020bde404c1,c_9cf7f631f662
49,t_003bb2871126,c_900e086f821e c_ec78b46e1cbe
55,t_0043301d4771,c_61f996ea544e c_677d2b86e021 c_7a34858ccb48 c...
...,...,...
61472,t_ffc71a181765,c_d7be46af5e2b c_ea92fdd9899a
61476,t_ffcb44cf3348,c_c4308e528ecb c_de4a12d54412
61484,t_ffd908252a7d,c_5d014c6f7def c_d84eedcb7d52
61510,t_fff7782561f4,c_bca8280a9ad1


In [140]:
comp_fbeta_score(target_df["content_id"], valid_pred_df["content_id"])

0.004674422870271427

In [139]:
comp_fbeta_score(target_df["content_id"], pred_df["content_id"])

0.004398162522812917

In [43]:
for i in [0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12]:
    valid_df["pred_prob"] = best_val_preds
    valid_df["pred"] = np.where(best_val_preds>i, 1, 0)
    valid_df = valid_df.sort_values("pred_prob", ascending=False)
    valid_df_ = valid_df[valid_df["pred"]==1]
    valid_pred_df = valid_df_.groupby("topic_id").agg(list).reset_index()
    valid_pred_df["content_id"] = valid_pred_df["content_id"].apply(lambda x: " ".join(x))
    print(comp_fbeta_score(target_df["content_id"], valid_pred_df["content_id"]))

0.6660931159683587
0.6660931159683587
0.6660931159683587
0.666075424941117
0.33690729948350856
0.041033282759774
0.1859188695289354
0.05862094567636588
0.01146702135847862
0.01213712710169587
0.06755882943225078
0.015993610780300576
0.01236158719035882


In [143]:
for i in [0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12]:
    valid_df["pred_prob"] = best_val_preds
    valid_df["pred"] = np.where(best_val_preds>i, 1, 0)
    valid_df = valid_df.sort_values("pred_prob", ascending=False)
    valid_df_ = valid_df[valid_df["pred"]==1]
    valid_pred_df = valid_df_.groupby("topic_id").agg(list).reset_index()
    valid_pred_df["content_id"] = valid_pred_df["content_id"].apply(lambda x: " ".join(x))
    print(comp_fbeta_score(target_df["content_id"], valid_pred_df["content_id"]))

0.23699841679304032
0.21679033014741544
0.1892149796675095
0.17483156934092592
0.16232434776417734
0.14847666119000952
0.1430110210715904
0.13446749024068697
0.12943977807615445
0.03181848039979413
0.008458565658880502
0.015799572599887752
0.026872859905546168


In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(valid_df["target"], valid_df["pred"])

array([[ 5874, 95046],
       [ 1087, 19433]])

In [146]:
from sklearn.metrics import confusion_matrix
confusion_matrix(valid_df["target"], valid_df["pred"])

array([[302096,  52142],
       [ 27909,   4874]])